# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-02-18 12:54:31.962335: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-18 12:54:31.992927: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure : 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [2]:
from gensim.models import Word2Vec

In [3]:
word2vec = Word2Vec(X_train)

Let's reuse the functions of the previous exercise to convert your training and test data into something you can fed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad` :
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from searching at errors too late and from letting them propagate through the entire notebook.

In [5]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you do something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [6]:
baseline = (len(y_train)-y_train.sum())/len(y_train)
baseline

0.506

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [7]:
X_train_pad.shape

(2500, 200, 100)

In [8]:
from tensorflow.keras import layers, models

model = models.Sequential()

model.add(layers.Masking(input_shape = (200,100)))
model.add(layers.LSTM(20))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = "accuracy")

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy with greatly depend on your training test corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 200, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 20)                9680      
_________________________________________________________________
dense (Dense)                (None, 10)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 9,901
Trainable params: 9,901
Non-trainable params: 0
_________________________________________________________________


In [10]:
X_train_pad

array([[[ 0.70608366,  0.00509747,  0.8455451 , ..., -0.65791166,
         -0.23037809, -1.4856297 ],
        [ 0.16735044, -0.21091281,  0.85518205, ..., -1.2850119 ,
          1.2496222 ,  0.33463225],
        [ 0.57348317,  0.00518854,  0.05248606, ..., -0.88482016,
         -0.5223239 ,  0.40033826],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.46885693,  1.2760179 ,  1.3749989 , ...,  0.4896467 ,
          0.32476133, -1.1086618 ],
        [-1.7429032 , -0.23128581,  0.28246394, ..., -0.91579926,
         -0.51742995, -0.99170727],
        [-1.0982168 ,  0.15054137,  0.9587878 , ..., -0.21286088,
         -0.66075015, -0.22578748],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 5, restore_best_weights=True)

model.fit(X_train_pad, y_train, validation_split=0.3, epochs = 100, batch_size = 16, callbacks = [es], workers = -1)

Epoch 1/100
110/110 [==============================] - 10s 69ms/step - loss: 0.6942 - accuracy: 0.5234 - val_loss: 0.6770 - val_accuracy: 0.5800
Epoch 2/100
110/110 [==============================] - 7s 60ms/step - loss: 0.6773 - accuracy: 0.5731 - val_loss: 0.6675 - val_accuracy: 0.5893
Epoch 3/100
110/110 [==============================] - 7s 62ms/step - loss: 0.6584 - accuracy: 0.6029 - val_loss: 0.6597 - val_accuracy: 0.5987
Epoch 4/100
110/110 [==============================] - 7s 62ms/step - loss: 0.6430 - accuracy: 0.6240 - val_loss: 0.6628 - val_accuracy: 0.6253
Epoch 5/100
110/110 [==============================] - 7s 63ms/step - loss: 0.6298 - accuracy: 0.6526 - val_loss: 0.6492 - val_accuracy: 0.6267
Epoch 6/100
110/110 [==============================] - 7s 61ms/step - loss: 0.6215 - accuracy: 0.6486 - val_loss: 0.6435 - val_accuracy: 0.6253
Epoch 7/100
110/110 [==============================] - 7s 64ms/step - loss: 0.6105 - accuracy: 0.6669 - val_loss: 0.6527 - val_accuracy

❓ **Question** ❓ Evaluate your model on the test set

In [12]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 1s 10ms/step - loss: 0.6295 - accuracy: 0.6456


[0.6295009255409241, 0.6456000208854675]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. On the other hand, let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that other have learnt? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this : 

In [13]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is the smaller (65 MB).

</details>

In [14]:
word2vec_transfer = api.load('glove-wiki-gigaword-50')

❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [19]:
word2vec_transfer.vector_size

50

In [26]:
len(word2vec_transfer.key_to_index)

400000

❓ **Question** ❓ Let's embedd `X_train` and `X_test`, as in the first question where we provided the functions to so ! (There is a slight different in the `embed_sentence_with_TF` function that we will not dig into)

In [27]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [29]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype = 'float32', padding = 'post', maxlen = 200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype = 'float32', padding = 'post', maxlen = 200)

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training could take some time here. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [34]:
word2vec_transfer['dog']

array([ 0.11008  , -0.38781  , -0.57615  , -0.27714  ,  0.70521  ,
        0.53994  , -1.0786   , -0.40146  ,  1.1504   , -0.5678   ,
        0.0038977,  0.52878  ,  0.64561  ,  0.47262  ,  0.48549  ,
       -0.18407  ,  0.1801   ,  0.91397  , -1.1979   , -0.5778   ,
       -0.37985  ,  0.33606  ,  0.772    ,  0.75555  ,  0.45506  ,
       -1.7671   , -1.0503   ,  0.42566  ,  0.41893  , -0.68327  ,
        1.5673   ,  0.27685  , -0.61708  ,  0.64638  , -0.076996 ,
        0.37118  ,  0.1308   , -0.45137  ,  0.25398  , -0.74392  ,
       -0.086199 ,  0.24068  , -0.64819  ,  0.83549  ,  1.2502   ,
       -0.51379  ,  0.04224  , -0.88118  ,  0.7158   ,  0.38519  ],
      dtype=float32)

In [35]:
X_train_pad_2.shape

(2500, 200, 50)

In [41]:
model2 = models.Sequential()

model2.add(layers.Masking(input_shape = (200,50)))
model2.add(layers.LSTM(20))
model2.add(layers.Dense(10, activation='relu'))
model2.add(layers.Dense(1, activation = 'sigmoid'))

model2.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = "accuracy")

In [42]:
model2.fit(X_train_pad_2, y_train,validation_split=0.3, epochs = 10, 
           batch_size = 16, workers = -1)

Epoch 1/10
110/110 [==============================] - 9s 66ms/step - loss: 0.6847 - accuracy: 0.5560 - val_loss: 0.6745 - val_accuracy: 0.5920
Epoch 2/10
110/110 [==============================] - 7s 63ms/step - loss: 0.6510 - accuracy: 0.6274 - val_loss: 0.6379 - val_accuracy: 0.6440
Epoch 3/10
110/110 [==============================] - 7s 61ms/step - loss: 0.6106 - accuracy: 0.6691 - val_loss: 0.6141 - val_accuracy: 0.6467
Epoch 4/10
110/110 [==============================] - 7s 61ms/step - loss: 0.5935 - accuracy: 0.6954 - val_loss: 0.5950 - val_accuracy: 0.7027
Epoch 5/10
110/110 [==============================] - 7s 61ms/step - loss: 0.5678 - accuracy: 0.7143 - val_loss: 0.6337 - val_accuracy: 0.6373
Epoch 6/10
110/110 [==============================] - 7s 62ms/step - loss: 0.5555 - accuracy: 0.7211 - val_loss: 0.5447 - val_accuracy: 0.7293
Epoch 7/10
110/110 [==============================] - 7s 63ms/step - loss: 0.5420 - accuracy: 0.7314 - val_loss: 0.5542 - val_accuracy: 0.7480

In [44]:
res = model2.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 70.240%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarder words that were not present more than a given number of time in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously